In [2]:
import pandas as pd
import selenium
import webdriver_manager
from datetime import datetime 

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support import expected_conditions as EC 
from time import sleep
import pandas as pd
from selenium import webdriver

from tqdm import tqdm
import os
from bs4 import BeautifulSoup
import requests
import re

import zipfile
import shutil

import os


import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

import json
# Ignorar warings
import warnings
warnings.filterwarnings("ignore")

import dotenv
from dotenv import load_dotenv
load_dotenv()

import psycopg2
from psycopg2 import sql
pd.set_option('display.max_columns', None)

import sys
sys.path.append("../")  

In [284]:
keywallapop= os.getenv("keywallapop")

Cuidado, de wallapop solo hay 20 llamadas mensuales

In [285]:

url = "https://wallapop3.p.rapidapi.com/search"

querystring = {"query":"nissan","minPrice":"2000","sort":"mostRecent"}

headers = {
	"x-rapidapi-key": keywallapop,
	"x-rapidapi-host": "wallapop3.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

resposa=response.json()

SOLO NOS INTESESA:  
- id (pasaré a numérico)  
- user_id  
- title  
- description  
- category_id (lo quiero para filtrar por 100(codigo correspondiente a coches)para que no me coja otros códigos que son de carroceria etc)  
- price (que está en euros)  
- location (por si las moscas)  
- capacidad de traslado (no creo que me lo quede)  
- refurbished(igual la mantengo pero los true los paso a 1 y los false a 0 porque en sql existe el bit)

In [ ]:
with open("../src", "w") as json_file:
    json.dump(resposa, json_file, indent=4)

In [346]:
len (resposa)

40

In [466]:
lista=[]
df_final=pd.DataFrame()
for i in range(0,40):
    id_wallapop=resposa[i]["id"]
    id_user=resposa[i]["user_id"]
    version=resposa[i]["title"]
    descripcion=resposa[i]["description"]
    id_categoria=resposa[i]["category_id"]
    precio=resposa[i]["price"]["amount"]
    latitud=resposa[i]["location"]["latitude"]
    longitud=resposa[i]["location"]["longitude"]
    ciudad=resposa[i]["location"]["city"]
    provincia=resposa[i]["location"]["region2"]
    envio_disponible=resposa[i]["shipping"]["item_is_shippable"]
    reparado=resposa[i]["is_refurbished"]["flag"]

    lista.append([id_wallapop,id_user,title,descripcion,id_category,precio,latitud, longitud,ciudad,provincia,envio_disponible,reparado] )
    df=pd.DataFrame(lista)
    df_final= pd.concat([df_final,df])

In [467]:

dicc_encabezados={0:"id_wallapop",
                  1: "id_user",
                  2: "version",
                  3: "descripcion",
                  4: "id_categoria",
                  5: "precio",
                  6: "latitud",
                  7: "longitud",
                  8: "ciudad",
                  9: "provincia",
                  10: "envio_disponible",
                  11: "reparado"}
df_final.columns= df_final.columns.map(dicc_encabezados)

In [468]:
df_final.sample(3)

,id_wallapop,id_user,version,descripcion,id_categoria,precio,latitud,longitud,ciudad,provincia,envio_disponible,reparado
7,mzn27nex87zn,mzng3427wg6n,Nissan Navara 2007,nissan patrol \n2.800 diesel \ngasta muy poco\...,100,5500,36.706282,-4.444297,Málaga,Málaga,False,False
0,8z8kp7plo8z3,xzo8xxq2l469,Nissan Navara 2007,Un HALLOWEEN de miedo en Flexicar: hasta un 20...,100,24990,42.140100,-0.405080,Huesca,Huesca,False,False
0,8z8kp7plo8z3,xzo8xxq2l469,Nissan Navara 2007,Un HALLOWEEN de miedo en Flexicar: hasta un 20...,100,24990,42.140100,-0.405080,Huesca,Huesca,False,False


In [469]:
df_final.shape

(820, 12)

ESTA LISTO PARA LA LIMPIEZA LO DE WALLAPOP

**Tratamiento de duplicados y nulos Wallapop**  
*NULOS*  

No hay nulos

*DUPLICADOS*  

Me he fijado que tiene muchos duplicados y que por alguna razon en la extraccion se han repetido con lo que sería necesario eliminarlos (o los users han subido el mismo anuncio muchas veces cosa que dudo porque wallapop no deja)

*DATOS INNECESARIOS*  

Eliminamos las columnas que no nos interesan y que aportan más problemas que información   

He intentado quedarme con el nombre de la version en una nueva columna pero hay demasiados anuncios que no ponen el nombre de la version entonces no me valía.


*Importante a tener en cuenta*  

Al final me quedo solo con la provincia de Madrid porque el resto del proyecto lo estoy haciendo en base a Madrid provincia


SOLO NOS INTESESA:  
- id (pasaré a numérico)  
- user_id  
- title  
- description  
- category_id (lo quiero para filtrar por 100(codigo correspondiente a coches)para que no me coja otros códigos que son de carroceria etc)  
- price (que está en euros)  
- location (por si las moscas)  
- capacidad de traslado (no creo que me lo quede)  
- refurbished(igual la mantengo pero los true los paso a 1 y los false a 0 porque en sql existe el bit pero no tiene valores true)

In [470]:
df_final.duplicated().sum()

np.int64(780)

In [471]:
df_final.drop_duplicates(inplace=True)

Ahora que ya he comprobado que solo hay categoría coche, la elimino

In [472]:
df_final["id_categoria"].unique()

array([100])

In [473]:
df_final["version"].unique()

array(['Nissan Navara 2007'], dtype=object)

In [474]:
df_final["envio_disponible"].unique()

array([False])

In [475]:
df_final["reparado"].unique()

array([False])

In [476]:
df_final=df_final.drop(columns= ["id_wallapop","id_user","id_categoria", "version", "envio_disponible", "reparado"])

In [477]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   descripcion  40 non-null     object 
 1   precio       40 non-null     int64  
 2   latitud      40 non-null     float64
 3   longitud     40 non-null     float64
 4   ciudad       40 non-null     object 
 5   provincia    40 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 2.2+ KB


In [478]:
df_final.sample(3)

,descripcion,precio,latitud,longitud,ciudad,provincia
39,"Vendo Nissan Navara D40, en muy buen estado in...",14000,41.093278,0.636733,Móra d'Ebre,Tarragona
18,Vehículo funcionando perfectamente. Cierre cen...,6200,38.573406,-0.653480,Castalla,Alacant/Alicante
4,"Bien cuidado, único propietario, siempre en ga...",2200,42.338660,1.402906,Adrall,Lleida


In [479]:
df_final=df_final[df_final["provincia"]=="Madrid"]
df_final.reset_index(drop=True, inplace=True)

Añado un id

In [480]:
df_final.index=df_final.index+1
df_final.reset_index(inplace= True)

In [481]:

dicc_encabezados0={"index":"id_wallapop",
                  "descripcion": "descripcion",
                  "precio": "precio",
                  "latitud": "latitud",
                  "longitud": "longitud",
                  "ciudad": "ciudad",
                  "provincia": "provincia"}

df_final.columns=df_final.columns.map(dicc_encabezados0)

In [482]:
df_final.to_csv("../datos/datos_wallapop.csv")
df_wallapop=pd.read_csv("../datos/datos_wallapop.csv", index_col= "Unnamed: 0" )

In [483]:
df_wallapop.head(4)

,id_wallapop,descripcion,precio,latitud,longitud,ciudad,provincia
0,1,nisan almera tino del 2004 ba por cadena le ve...,2200,40.301646,-3.441828,Arganda del Rey,Madrid
1,2,vendo mi coche en perfecto estado de motor y d...,6500,40.620256,-3.910636,Hoyo de Manzanares,Madrid
2,3,coche usado por mi mujer para ir a trabajar.\n...,11500,40.494190,-3.961537,Villanueva del Pardillo,Madrid
3,4,"se vende nissan patrol k260 embrague nuevo , b...",4500,40.419014,-3.707443,Madrid,Madrid


API seguridad del modelo

In [236]:
url_anio = f"https://api.nhtsa.gov/SafetyRatings/modelyear/2013/make/nissan"
response3 = requests.get(url_anio)
if response3.status_code == 200:
    modelos = [diccio["Model"] for diccio in response3.json().get("Results", [])]
else:
    print(f"Error: {response3.status_code} - {response3.text}")
    


In [ ]:


df_ratings = pd.DataFrame()

for anio in range(2011, 2026):
    url_anio = f"https://api.nhtsa.gov/SafetyRatings/modelyear/{anio}/make/nissan"
    response3 = requests.get(url_anio)
    
    #verifica si no da error
    if response3.status_code == 200:
        try:
            modelos = [diccio["Model"] for diccio in response3.json().get("Results", [])]
        except (KeyError, ValueError) as e:
            print(f"Error al obtener modelos para el año {anio}: {e}")
            continue
    else:
        print(f"Error en la solicitud para el año {anio}: {response3.status_code}")
        continue

    modelos_ids = []
    for modelo in modelos:
        url3 = f"https://api.nhtsa.gov/SafetyRatings/modelyear/{anio}/make/nissan/model/{modelo}"
        response = requests.get(url3)

        if response.status_code == 200:
            try:
                modelos_ids.extend([diccio["VehicleId"] for diccio in response.json().get("Results", [])])
            except (KeyError, ValueError) as e:
                print(f"Error al obtener VehicleId para modelo {modelo} en el año {anio}: {e}")
                continue
        else:
            print(f"Error en la solicitud para modelo {modelo}, año {anio}: {response.status_code}")
            continue

    #conseguir ratings con modelos_ids
    for modelo_id in modelos_ids:
        url2 = f"https://api.nhtsa.gov/SafetyRatings/VehicleId/{modelo_id}"
        response = requests.get(url2)

        if response.status_code == 200:
            try:
                df = pd.DataFrame(response.json().get("Results", []))
                df_ratings = pd.concat([df_ratings, df], ignore_index=True)
            except (KeyError, ValueError) as e:
                print(f"Error al obtener ratings para VehicleId {modelo_id}: {e}")
        else:
            print(f"Error en la solicitud para VehicleId {modelo_id}: {response.status_code}")


In [242]:
df_ratings.sample(4)

,VehiclePicture,OverallRating,OverallFrontCrashRating,FrontCrashDriversideRating,FrontCrashPassengersideRating,OverallSideCrashRating,SideCrashDriversideRating,SideCrashPassengersideRating,combinedSideBarrierAndPoleRating-Front,combinedSideBarrierAndPoleRating-Rear,sideBarrierRating-Overall,RolloverRating,RolloverRating2,RolloverPossibility,RolloverPossibility2,dynamicTipResult,SidePoleCrashRating,NHTSAElectronicStabilityControl,NHTSAForwardCollisionWarning,NHTSALaneDepartureWarning,ComplaintsCount,RecallsCount,InvestigationCount,ModelYear,Make,Model,VehicleDescription,VehicleId,FrontCrashPicture,FrontCrashVideo,SideCrashPicture,SideCrashVideo,SidePolePicture,SidePoleVideo
0,https://static.nhtsa.gov/images/vehicles/7276_...,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,0.000,0.0,,Not Rated,Standard,No,No,40,1,1,2011,NISSAN,370Z,2011 Nissan 370Z 2 DR RWD,355,NaN,NaN,NaN,NaN,NaN,NaN
474,https://static.nhtsa.gov/images/vehicles/52562...,5,4,5,4,5,4,5,5,5,5,5,Not Rated,0.092,0.0,No Tip,5,Standard,Standard,Standard,14,1,0,2024,NISSAN,ALTIMA,2024 Nissan Altima 4 DR AWD,19561,https://static.nhtsa.gov/crashTest/images/2024...,https://static.nhtsa.gov/crashTest/videos/2024...,https://static.nhtsa.gov/crashTest/images/2024...,https://static.nhtsa.gov/crashTest/videos/2024...,https://static.nhtsa.gov/crashTest/images/2024...,https://static.nhtsa.gov/crashTest/videos/2024...
289,https://static.nhtsa.gov/images/vehicles/12855...,4,4,4,5,5,5,5,5,5,5,4,Not Rated,0.198,0.0,No Tip,5,Standard,No,No,65,4,2,2018,NISSAN,TITAN,2018 Nissan Titan Crew Cab RWD Early Release,12697,https://static.nhtsa.gov/crashTest/images/2018...,https://static.nhtsa.gov/crashTest/videos/2018...,https://static.nhtsa.gov/crashTest/images/2018...,https://static.nhtsa.gov/crashTest/videos/2018...,https://static.nhtsa.gov/crashTest/images/2018...,https://static.nhtsa.gov/crashTest/videos/2018...
281,https://static.nhtsa.gov/images/vehicles/12937...,4,4,5,2,5,5,5,5,5,5,4,Not Rated,0.143,0.0,No Tip,5,Standard,Optional,No,67,2,0,2018,NISSAN,ROGUE SPORT,2018 Nissan Rogue Sport SUV AWD,12773,https://static.nhtsa.gov/crashTest/images/2018...,https://static.nhtsa.gov/crashTest/videos/2018...,https://static.nhtsa.gov/crashTest/images/2018...,https://static.nhtsa.gov/crashTest/videos/2018...,https://static.nhtsa.gov/crashTest/images/2018...,https://static.nhtsa.gov/crashTest/videos/2018...


In [239]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 34 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   VehiclePicture                          422 non-null    object 
 1   OverallRating                           525 non-null    object 
 2   OverallFrontCrashRating                 525 non-null    object 
 3   FrontCrashDriversideRating              525 non-null    object 
 4   FrontCrashPassengersideRating           525 non-null    object 
 5   OverallSideCrashRating                  525 non-null    object 
 6   SideCrashDriversideRating               525 non-null    object 
 7   SideCrashPassengersideRating            525 non-null    object 
 8   combinedSideBarrierAndPoleRating-Front  525 non-null    object 
 9   combinedSideBarrierAndPoleRating-Rear   525 non-null    object 
 10  sideBarrierRating-Overall               525 non-null    object

In [268]:
df_ratings.to_csv("../datos/datos_seguridad.csv")
df_seguridad=pd.read_csv("../datos/datos_seguridad.csv", index_col= "Unnamed: 0" )

ESTÁ listo para la limpieza lo de SEGURIDAD

**Tratamiento de duplicados y nulos Seguridad**  
*NULOS*  

Había muchos Not Rated que convertí en nulos para eliminarlos porque si no tiene rating, no nos aporta nada de valor 

*DUPLICADOS*  

No tiene duplicados


*DATOS INNECESARIOS*  

Eliminamos las columnas que no nos interesan y que aportan más problemas que información 


In [269]:
df_seguridad=df_seguridad.drop(columns= ["VehiclePicture","FrontCrashDriversideRating","FrontCrashPassengersideRating", "SideCrashDriversideRating", "SideCrashPassengersideRating", "combinedSideBarrierAndPoleRating-Front", "combinedSideBarrierAndPoleRating-Rear", "RolloverRating2", "RolloverPossibility2", "dynamicTipResult", "FrontCrashPicture", "FrontCrashVideo", "SideCrashPicture", "SideCrashVideo", "SidePolePicture", "SidePoleVideo"])

In [281]:
df_seguridad.head(4)

,id_seguridad,evaluacion_general,evaluacion_choque_frontal,evaluacion_choque_lateral,evaluacion_barrera_lateral,evaluacion_vuelco,probabilidad_vuelco,evaluacion_choque_columna,estabilidad_electronica,aviso_choque_frontal,aviso_desvio_carril,quejas,retiradas,investigaciones,anio,marca,version,descripcion,vehiculo_id
0,1,4,4,5,4,4,0.105,5,Standard,No,No,236,1,0,2011,NISSAN,ALTIMA,2011 Nissan Altima 4 DR FWD,357
1,2,4,4,5,4,4,0.105,5,Standard,No,No,240,1,1,2011,NISSAN,ALTIMA HEV,2011 Nissan Altima HEV 4 DR FWD,359
2,3,5,4,5,5,4,0.110,4,Standard,No,No,93,0,3,2011,NISSAN,LEAF,2011 Nissan Leaf 5 HB FWD,367
3,4,4,4,5,5,4,0.185,4,Standard,No,No,145,0,0,2011,NISSAN,MURANO,2011 Nissan Murano SUV AWD,654


In [275]:
df_seguridad.replace("Not Rated", pd.NA, inplace=True)

columnas_rating = ["OverallRating", "OverallFrontCrashRating", "OverallSideCrashRating","sideBarrierRating-Overall", "RolloverRating", "SidePoleCrashRating"]
df_seguridad.dropna(subset=columnas_rating, inplace=True)
df_seguridad.reset_index(drop= True, inplace=True)

In [276]:
df_seguridad[columnas_rating] = df_seguridad[columnas_rating].astype(int)

In [277]:
df_seguridad.duplicated().sum()

np.int64(0)

In [278]:
df_seguridad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OverallRating                    249 non-null    int64  
 1   OverallFrontCrashRating          249 non-null    int64  
 2   OverallSideCrashRating           249 non-null    int64  
 3   sideBarrierRating-Overall        249 non-null    int64  
 4   RolloverRating                   249 non-null    int64  
 5   RolloverPossibility              249 non-null    float64
 6   SidePoleCrashRating              249 non-null    int64  
 7   NHTSAElectronicStabilityControl  249 non-null    object 
 8   NHTSAForwardCollisionWarning     249 non-null    object 
 9   NHTSALaneDepartureWarning        249 non-null    object 
 10  ComplaintsCount                  249 non-null    int64  
 11  RecallsCount                     249 non-null    int64  
 12  InvestigationCount    

Añado un id

In [279]:
df_seguridad.index= df_seguridad.index+1
df_seguridad.reset_index(inplace=True)

In [280]:
dicc_encabezados={
    "index": "id_seguridad",
    "OverallRating": "evaluacion_general",
    "OverallFrontCrashRating": "evaluacion_choque_frontal",
    "OverallSideCrashRating": "evaluacion_choque_lateral",
    "sideBarrierRating-Overall": "evaluacion_barrera_lateral",
    "RolloverRating": "evaluacion_vuelco",
    "RolloverPossibility": "probabilidad_vuelco",
    "SidePoleCrashRating": "evaluacion_choque_columna",
    "NHTSAElectronicStabilityControl": "estabilidad_electronica",
    "NHTSAForwardCollisionWarning": "aviso_choque_frontal",
    "NHTSALaneDepartureWarning": "aviso_desvio_carril",
    "ComplaintsCount": "quejas",
    "RecallsCount": "retiradas",
    "InvestigationCount": "investigaciones",
    "ModelYear": "anio",
    "Make": "marca",
    "Model": "version",
    "VehicleDescription": "descripcion",
    "VehicleId": "vehiculo_id"
}
            
df_seguridad.columns=df_seguridad.columns.map(dicc_encabezados)

In [282]:
df_seguridad.to_csv("../datos/datos_seguridad.csv")

Empezamos con web scrapping de Quecochemecompro

Este funciona sin duplicados(Primera tabla solo porque es donde están los modelos más simples y el cliente al que estamos intentando informar prioriza mucho el ahorro)

In [141]:
url_nuevo = "https://www.quecochemecompro.com/"

driver=webdriver.Chrome()
driver.maximize_window()
driver.get(url_nuevo)

#acepto cookies
driver.find_element("css selector", "#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-y6krop").click()
sleep(2)
#me meto en nuevos
driver.find_element("css selector", "#header-menu > li:nth-child(1) > a").click()


sleep(2)
#click en el buscador 
driver.find_element("css selector", "#header > div.tw-container.flex.gap-4 > div > div.hidden.lg\:flex.w-60.ml-auto.mr-8.gap-4.items-center > form").click()
sleep(2)
#meto nissan en el buscador
driver.find_element("css selector", "#header > div.tw-container.flex.gap-4 > div > div.hidden.lg\:flex.w-60.ml-auto.mr-8.gap-4.items-center > form > input").send_keys("nissan", Keys.ENTER)

#saco los nombres para meterlos en una lista

sopa_nuevos = BeautifulSoup(driver.page_source, 'html.parser')
lista_nombre_modelo=[]
for modelo in sopa_nuevos.find_all("li", {"class":"mx-auto w-full min-w-[20rem]"}):
    tag_a = modelo.find("a", title=True)
    if tag_a:
        lista_nombre_modelo.append(tag_a["title"])


df_datos_modelos = pd.DataFrame()
for n_modelo in range(1, len(lista_nombre_modelo) + 1):
    sleep(2)
    # Hacer click en el modelo específico
    driver.find_element("css selector", f"#results > ul > li:nth-child({n_modelo}) > div > div.relative.z-10 > div.flex.justify-between.items-end.py-3.leading-5.text-17 > div:nth-child(1) > a").click()
    sopa_tabla = BeautifulSoup(driver.page_source, 'html.parser')
    sleep(2)
    df_tablas=pd.DataFrame()
    
    try:  # Encontrar la tabla de datos
        
        sleep(2) #hace click por si hay más tablas

        try:
            driver.find_element("css selector", f"#versions > div > div:nth-child(2) > table > tfoot > tr > td > ul > li:nth-child(2) > a").click()
            
        except:
            # Si no encuentra el selector con nth-child(2), intenta con nth-child(3)
            
            driver.find_element("css selector", f"#versions > div > div:nth-child(3) > table > tfoot > tr > td > ul > li:nth-child(2) > a").click()
            
        tabla = sopa_tabla.find("table")  #a lo mejor no me coge en algunas porque hay otra tabla                
        
        encabezados = [th.get_text(strip=True) for th in tabla.find_all("thead")[0].find_all("th")]

        filas = []
        for tr in tabla.find_all("tbody")[0].find_all("tr"):
            filas.append([td.get_text(strip=True) for td in tr.find_all("td")])

        df_tabla = pd.DataFrame(filas, columns=encabezados)
        df_tabla["Modelo"] = lista_nombre_modelo[n_modelo - 1]
        df_tablas= pd.concat([df_tablas, df_tabla])




        df_datos_modelos= pd.concat([df_datos_modelos, df_tablas], ignore_index=True)

    except:
        
        driver.back()
        continue
        
    driver.back()    




In [142]:
df_datos_modelos.sample(3)

,Versión,Etiqueta,Puertas,CV,Consumo,Motor,PVP,Oferta,Modelo,Autonomía
5,NISSAN QASHQAI (O)1.3 DIG-T MHEV 116KW (158CV)...,,5,158,"6,3 l/100Km",MHEV Gasolina,35.800 €,Pedir oferta,Nissan Qashqai,NaN
23,NISSAN JUKE (O)1.0 DIG-T 84KW (114CV) N-CONNECTA,,5,114,"5,8 l/100Km",Gasolina,26.975 €,Pedir oferta,Nissan Juke,NaN
7,NISSAN QASHQAI 1.3 DIG-T MHEV 116KW (158CV) N-...,,5,158,"6,4 l/100Km",MHEV Gasolina,37.950 €,Pedir oferta,Nissan Qashqai,NaN


In [222]:
df_datos_modelos.to_csv("../datos/datos_modelos_nuevos.csv")
df_datos_modelos_nuevos=pd.read_csv("../datos/datos_modelos_nuevos.csv", index_col= "Unnamed: 0" )

Coches nuevos listos para limpiar

**Tratamiento de duplicados y nulos Nuevos**  
*NULOS*  

Hay nulos en consumo y autonomía asique decido meter los valores de autonomia en los nan de consumo (porque corresponden a los coches electricos) y elimino la columna autonomía.  

Finalmente nos quedamos sin ningun nulo sin necesidad de hacer un dropna

*DUPLICADOS*  

No tiene duplicados


*DATOS INNECESARIOS*  

También elimino la columna etiqueta y oferta ya que no se ofrecen información relevante  

En la columna de Consumo me he fijado que hay 5 valores con "-" lo que no me permite formatear esa columna(dado que son tan pocos, voy a eliminarlos porque al tener la autonomía en la misma columna la media no es representativa)

In [234]:
df_datos_modelos_nuevos.sample(5)

,id_modelo_nuevo,version,puertas,potencia,consumo,motor,precio,modelo
2,3,NISSAN QASHQAI 1.3 DIG-T MHEV 116KW (158CV) AC...,5,158,6.3,MHEV Gasolina,34300,Nissan Qashqai
45,46,NISSAN X-TRAIL (O)1.5T MHEV 120KW (163CV) CVT ...,5,163,7.2,MHEV Gasolina,44550,Nissan X-Trail
87,92,NISSAN PRIMASTAR 2.0DCI 110KW (150CV) ACENTA L...,4,150,7.0,Diesel,45475,Nissan Primastar
62,67,NISSAN TOWNSTAR 1.3 TCE 96KW (130CV) TEKNA,5,130,6.8,Gasolina,37103,Nissan Townstar
49,51,NISSAN X-TRAIL 1.5 HEV E-POWER 204 N-CONNECTA,5,204,5.8,HEV Gasolina,45000,Nissan X-Trail híbrido


In [224]:
df_datos_modelos_nuevos["Consumo"] = df_datos_modelos_nuevos["Consumo"].fillna(df_datos_modelos_nuevos["Autonomía"])


In [225]:
df_datos_modelos_nuevos=df_datos_modelos_nuevos.drop(columns= ["Etiqueta","Oferta","Autonomía"])

In [226]:
df_datos_modelos_nuevos["Consumo"]=df_datos_modelos_nuevos["Consumo"].str.replace("l/100Km", "").str.replace(",",".").str.replace(" Km", "")
df_datos_modelos_nuevos["PVP"]=df_datos_modelos_nuevos["PVP"].str.replace("€", "").str.replace(".", "")

In [227]:
df_datos_modelos_nuevos.duplicated().sum()

np.int64(0)

In [228]:
df_datos_modelos_nuevos=df_datos_modelos_nuevos[df_datos_modelos_nuevos["Consumo"]!= "-"]

In [229]:
df_datos_modelos_nuevos["PVP"]=df_datos_modelos_nuevos["PVP"].astype(int)
df_datos_modelos_nuevos["Consumo"]=df_datos_modelos_nuevos["Consumo"].astype(float)

Añado un id

In [230]:
df_datos_modelos_nuevos.index= df_datos_modelos_nuevos.index+1
df_datos_modelos_nuevos.reset_index(inplace=True)

In [231]:
dicc_encabezados1={"index":"id_modelo_nuevo",
                    "Versión":"version",
                    "Puertas": "puertas",
                    "CV": "potencia",
                    "Consumo": "consumo",
                    "Motor": "motor",
                    "PVP": "precio",
                    "Modelo": "modelo"
                  }
            
df_datos_modelos_nuevos.columns=df_datos_modelos_nuevos.columns.map(dicc_encabezados1)

In [232]:
df_datos_modelos_nuevos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_modelo_nuevo  107 non-null    int64  
 1   version          107 non-null    object 
 2   puertas          107 non-null    int64  
 3   potencia         107 non-null    int64  
 4   consumo          107 non-null    float64
 5   motor            107 non-null    object 
 6   precio           107 non-null    int64  
 7   modelo           107 non-null    object 
dtypes: float64(1), int64(4), object(3)
memory usage: 6.8+ KB


In [233]:
df_datos_modelos_nuevos.to_csv("../datos/datos_modelos_nuevos.csv")

Datos de modelos nuevos ya están formateados

Alternativa 2º mano

In [58]:
url_alterantiva= "https://www.autoscout24.es/"
driver=webdriver.Chrome()
driver.maximize_window()
driver.get(url_alterantiva)

#acepto cookies
driver.find_element("css selector", "#as24-cmp-popup > div > div._acceptance-buttons_1lphq_85 > button._consent-accept_1lphq_114").click()

#despliego menu
driver.find_element("css selector", "#make").click()
sleep(2)
#elijo nissan
driver.find_element("css selector", "#make > optgroup:nth-child(3) > option:nth-child(172)").click()
sleep(2)
#le da a buscar
driver.find_element("css selector", "#search-mask-search-cta").click()
sleep(2)

#que nos de solo madrid
driver.find_element("css selector", "#__next > div > div > div.ListPage_wrapper__vFmTi > div.ListPage_container__Optya > div:nth-child(2) > aside > div > div:nth-child(2) > div:nth-child(1) > div > div.Location_wrapper__mvayy > div:nth-child(2) > div > div > div.input-wrapper > div > button > svg").click()
sleep(2)


df_completo=pd.DataFrame()
for pag in range(1,21):
    df=pd.DataFrame()
    url_pag=f"https://www.autoscout24.es/lst/nissan?atype=C&cy=E&damaged_listing=exclude&desc=0&lat=40.435712&lon=-3.6405248&page={pag}&powertype=kw&search_id=1yf3g5xh5i1&sort=standard&source=listpage_pagination&ustate=N%2CU&zipr=200"
    driver.get(url_pag)
    sopa_alternativa = BeautifulSoup(driver.page_source, 'html.parser')
    sleep(2)
    #conseguir nombre
    lista_nombres = sopa_alternativa.findAll("h2")
    todos_nombres=[nombre.getText().replace("\xa0", " ") for nombre in lista_nombres]
    df_nombre=pd.DataFrame(todos_nombres)


    #conseguir datos
    lista_datos= sopa_alternativa.findAll("div", {"class":"VehicleDetailTable_container__XhfV1"})
    todos_datos = [re.sub(r'(Manual|Automático)\d{2}/(\d{4})(Diésel|Gasolina|Electro/Gasolina|Eléctrico)\d*', r'\1 \2 \3', dato.getText()).replace("km", " ").replace("CV", "").replace("kW", "").replace(".", "").replace("(", "").replace(")", "").split() for dato in lista_datos]
    df_dato=pd.DataFrame(todos_datos)


    #Conseguir precio
    lista_precios= sopa_alternativa.findAll("p", {"class":"Price_price__APlgs PriceAndSeals_current_price__ykUpx"})
    todos_precios=[precio.getText().replace("€ ","").replace(",-","").replace(".","") for precio in lista_precios]
    df_precio=pd.DataFrame(todos_precios)

    df= pd.concat([df,df_nombre,df_dato,df_precio], axis=1, ignore_index=True)
    df_completo=pd.concat([df_completo, df], axis=0, ignore_index=True)


In [59]:
df_completo.sample(5)

,0,1,2,3,4,5,6,7,8,9,10
200,Nissan Micra 1.2 Tekna Premium,94311,Manual,2011,Gasolina,80,7990,NaN,NaN,NaN,NaN
51,Nissan Micra Acenta IG-T 92cv. *IVA deducible*...,30000,Manual,2022,Gasolina,92,12990,NaN,NaN,NaN,NaN
260,Nissan Qashqai 1.3 DIG-T mHEV 12V Acenta 4x2 1...,19161,Manual,2023,Gasolina,140,21636,NaN,NaN,NaN,NaN
22,Nissan Micra IG-T Acenta 92,26131,Manual,2023,Gasolina,92,11790,NaN,NaN,NaN,NaN
145,Nissan Qashqai 1.5dCi Acenta 4x2,270000,Manual,2007,Diésel,106,None,None,None,None,5800


**Tratamiento de duplicados y nulos Segunda Mano**  
*NULOS*  

Hay nulos porque se crean algunas columnas por datos que están formateados distinto. Puedo permitirme eliminar las columnas casi llenas de nulos y luego eliminar los restantes para que los 367 que se queden, tengan datos.  

*DUPLICADOS*  

Decido no eliminarlos porque el hecho de que se repitan nos muestra que coches se están vendiendo más


*DATOS INNECESARIOS*  

En la columna de precio se cuela Ocasión como si fuera un precio lo que me dificultaría pasar al formato correcto por ello como apenas son 10 los elimino

In [60]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       420 non-null    object
 1   1       400 non-null    object
 2   2       400 non-null    object
 3   3       400 non-null    object
 4   4       400 non-null    object
 5   5       398 non-null    object
 6   6       369 non-null    object
 7   7       9 non-null      object
 8   8       9 non-null      object
 9   9       9 non-null      object
 10  10      40 non-null     object
dtypes: object(11)
memory usage: 36.2+ KB


In [74]:
df_completo.duplicated().sum()

np.int64(52)

In [75]:
df_completo[df_completo.duplicated()].head()

,title,kilometraje,tipo,anio,fuel,potencia,precio
19,Nissan Pulsar 1.2 DIG-T Visia,93103,Manual,2016,Gasolina,116,10300
26,Nissan Pulsar 1.2 DIG-T Visia,93103,Manual,2016,Gasolina,116,10300
38,Nissan Qashqai 1.3 DIG-T mHEV 12V N-Connecta 4...,34763,Manual,2023,Gasolina,140,23545
74,Nissan NV200 15dCI COMFORT FURGÓN,64635,Manual,2018,Diésel,90,-/-Ocasión-
92,Nissan Qashqai DIG-T 103kW (140CV) mHEV 4x2 Ac...,15000,Manual,2023,Electro/Gasolina,140,23630


In [61]:
df_completo=df_completo.drop(columns= [7,8,9,10])

In [63]:
df_completo.dropna(inplace=True)

In [64]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 367 entries, 0 to 418
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       367 non-null    object
 1   1       367 non-null    object
 2   2       367 non-null    object
 3   3       367 non-null    object
 4   4       367 non-null    object
 5   5       367 non-null    object
 6   6       367 non-null    object
dtypes: object(7)
memory usage: 22.9+ KB


In [66]:
dicc_encabezados2={0:"title",
                  1: "kilometraje",
                  2: "tipo",
                  3: "anio",
                  4: "fuel",
                  5: "potencia",
                  6: "precio"}
df_completo.columns=df_completo.columns.map(dicc_encabezados2)

In [67]:
df_completo.sample(3)

,title,kilometraje,tipo,anio,fuel,potencia,precio
387,Nissan Qashqai 1.3 DIG-T mHEV 12V Acenta 4x2 1...,10678,Manual,2023,Gasolina,140,21990
33,Nissan Qashqai 1.3 DIG-T N-Style 4x2 103kW,64634,Automático,2021,Gasolina,140,18190
39,Nissan Qashqai 1.3 DIG-T mHEV 12V N-Connecta 4...,37544,Manual,2023,Gasolina,140,23545


Limpiamos los valores que no correspondan (En precio se han colado ocasion )

In [86]:

df_completo = df_completo[~df_completo["precio"].str.contains("-/-Ocasión", na=False)]


In [89]:
# df_completo["precio"].unique() Lo comento para que no moleste

In [90]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 358 entries, 0 to 418
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        358 non-null    object
 1   kilometraje  358 non-null    object
 2   tipo         358 non-null    object
 3   anio         358 non-null    object
 4   fuel         358 non-null    object
 5   potencia     358 non-null    object
 6   precio       358 non-null    object
dtypes: object(7)
memory usage: 22.4+ KB


In [212]:
df_completo.to_csv("../datos/datos_segunda_mano.csv")
df_segunda_mano=pd.read_csv("../datos/datos_segunda_mano.csv", index_col= "Unnamed: 0")

Aquí formateamos

In [213]:
df_segunda_mano["kilometraje"]=df_segunda_mano["kilometraje"].astype(int)
df_segunda_mano["potencia"]=df_segunda_mano["potencia"].astype(int)
df_segunda_mano["precio"]=df_segunda_mano["precio"].astype(int)
df_segunda_mano["anio"]=df_segunda_mano["anio"].astype(int)


In [214]:
df_segunda_mano.info()

<class 'pandas.core.frame.DataFrame'>
Index: 358 entries, 0 to 418
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        358 non-null    object
 1   kilometraje  358 non-null    int64 
 2   tipo         358 non-null    object
 3   anio         358 non-null    int64 
 4   fuel         358 non-null    object
 5   potencia     358 non-null    int64 
 6   precio       358 non-null    int64 
dtypes: int64(4), object(3)
memory usage: 22.4+ KB


Añado un id

In [215]:
df_segunda_mano.index= df_segunda_mano.index+1
df_segunda_mano.reset_index(inplace=True)

In [216]:
dicc_encabezados3={"index":"id_segunda_mano",
                    "title":"version",
                    "kilometraje": "kilometraje",
                    "tipo": "tipo",
                    "anio": "anio",
                    "fuel": "motor",
                    "potencia": "potencia",
                    "precio": "precio"
                  }
            
df_segunda_mano.columns=df_segunda_mano.columns.map(dicc_encabezados3)

In [217]:
df_segunda_mano.head()

,id_segunda_mano,version,kilometraje,tipo,anio,motor,potencia,precio
0,1,Nissan Qashqai 1.2 DIG-T Acenta 4x2,121649,Manual,2015,Gasolina,116,11480
1,2,Nissan Juke 1.5dCi Visia,260000,Manual,2012,Diésel,110,6500
2,3,Nissan Qashqai DIG-T 103kW (140CV) mHEV 4x2 Ac...,11290,Manual,2023,Electro/Gasolina,140,22000
3,4,Nissan Juke 1.6 N-Connecta 4x2 112,46287,Manual,2019,Gasolina,113,11790
4,5,Nissan Qashqai 1.6dCi Visia 4x2,178000,Manual,2018,Diésel,131,13900


In [218]:
df_segunda_mano.to_csv("../datos/datos_segunda_mano.csv")

Datos de segunda mano ya están formateados